In [1]:
%config IPCompleter.use_jedi = False

In [2]:
import pandas as pd
import numpy as np
import math

In [3]:
parties_df_raw = pd.read_html('https://ru.wikipedia.org/wiki/%D0%9F%D0%B0%D1%80%D0%BB%D0%B0%D0%BC%D0%B5%D0%BD%D1%82%D1%81%D0%BA%D0%B8%D0%B5_%D0%B2%D1%8B%D0%B1%D0%BE%D1%80%D1%8B_%D0%B2_%D0%91%D0%BE%D0%BB%D0%B3%D0%B0%D1%80%D0%B8%D0%B8_(2017)')[4]

In [4]:
parties_df_raw = parties_df_raw.iloc[:5]

In [185]:
election_dict = {x[0]:int(x[1]) for x in list(parties_df_raw[['Партия',
                                                         'Мест']].\
                                         set_index('Партия').\
                                         to_records())}

In [186]:
election_dict

{'ГЕРБ': 95,
 'БСП за Болгарию': 80,
 'Объединённые патриоты': 27,
 'Движение за права и свободы': 26,
 'Воля': 12}

In [132]:
election_dict = {'Democratic Party': 55,
                 'Party of Industrialists and Entrepreneurs': 11,
                 'Agrarian Party': 11,
                 'Independents': 48}

In [187]:
assert sum(list(election_dict.values()))==240
# проверка

In [188]:
QUOTA = math.ceil((240)/2)+1

In [189]:
QUOTA

121

In [242]:
from itertools import combinations

# https://stackoverflow.com/a/24402296
combinations_ = sum([list(map(list, combinations(election_dict, i))) for i in range(len(election_dict) + 1)], [])

In [243]:
combinations_clean = [combination for combination in combinations_ if len(combination)>1]

In [192]:
def get_vote_sum(combination):
    vote_sum = 0
    for party in combination:
        vote_sum += election_dict[party]
    return vote_sum

In [193]:
winning_coalitions = [combination for combination in combinations_clean if get_vote_sum(combination)>=QUOTA]

In [194]:
def is_pivotal(party, coalition):
    if party in coalition:
        sum_wout_party = sum([election_dict[element] for element in [prt for prt in coalition if prt != party]])
        if sum_wout_party < QUOTA:
            return True
        else:
            return False
    else:
        return False

In [195]:
pivotal = {}

for party in list(election_dict.keys()):
    res_list = []
    for coalition in winning_coalitions:
        res_list.append(is_pivotal(party, coalition))
    pivotal[party] = sum(res_list)

In [196]:
pivotal

{'ГЕРБ': 12,
 'БСП за Болгарию': 4,
 'Объединённые патриоты': 4,
 'Движение за права и свободы': 4,
 'Воля': 0}

In [197]:
sum_pivot = sum(list(pivotal.values()))

In [198]:
pivotal_list = list()

for coalition in winning_coalitions:
    coal_list = list()
    for party in coalition:
        coal_list.append(is_pivotal(party, coalition))
    pivotal_list.append(coal_list)

In [199]:
def is_min_winning(coalition):
    bool_list = list()
    for party in coalition:
        bool_list.append(is_pivotal(party, coalition))
    return all(bool_list)

In [200]:
min_winning = [coalition for coalition in winning_coalitions if is_min_winning(coalition)]

In [296]:
min_winning

[['ГЕРБ', 'БСП за Болгарию'],
 ['ГЕРБ', 'Объединённые патриоты'],
 ['ГЕРБ', 'Движение за права и свободы'],
 ['БСП за Болгарию', 'Объединённые патриоты', 'Движение за права и свободы']]

In [201]:
sum_pivotal_hpi = sum([len(coalition) for coalition in min_winning])

In [202]:
sum_pivotal_hpi

9

In [203]:
min_winning

[['ГЕРБ', 'БСП за Болгарию'],
 ['ГЕРБ', 'Объединённые патриоты'],
 ['ГЕРБ', 'Движение за права и свободы'],
 ['БСП за Болгарию', 'Объединённые патриоты', 'Движение за права и свободы']]

In [204]:
def hpi(party, min_winning):
    coal_count = 0
    for coalition in min_winning:
        if party in coalition:
            coal_count += 1
    return coal_count/sum([len(coalition) for coalition in min_winning])

In [205]:
# Holler-Packel index

In [206]:
for party in list(election_dict.keys()):
    print(party, hpi(party, min_winning))

ГЕРБ 0.3333333333333333
БСП за Болгарию 0.2222222222222222
Объединённые патриоты 0.2222222222222222
Движение за права и свободы 0.2222222222222222
Воля 0.0


In [207]:
len(winning_coalitions)

16

In [208]:
# Coleman index

In [209]:
def coleman(party, winning_coalitions, pivotal):
    return pivotal[party]/len(winning_coalitions)

In [210]:
len(winning_coalitions)

16

In [211]:
for party in list(election_dict.keys()):
    print(party, coleman(party, winning_coalitions, pivotal))

ГЕРБ 0.75
БСП за Болгарию 0.25
Объединённые патриоты 0.25
Движение за права и свободы 0.25
Воля 0.0


In [212]:
# Johnstone

In [229]:
def johnstone(party, winning_coalitions):
    party_ji = 0
    for coalition in winning_coalitions:
        if party in coalition and is_pivotal(party, coalition):
            party_ji += 1/sum([is_pivotal(party, coalition) for party in coalition])
    return(party, party_ji)

In [230]:
ji_dict = {johnstone(party, winning_coalitions)[0]: johnstone(party, winning_coalitions)[1] for party in list(election_dict.keys())}

In [231]:
ji_dict

{'ГЕРБ': 9.0,
 'БСП за Болгарию': 1.6666666666666665,
 'Объединённые патриоты': 1.6666666666666665,
 'Движение за права и свободы': 1.6666666666666665,
 'Воля': 0}

In [234]:
for party in list(ji_dict.keys()):
    print(party, ji_dict[party]/sum(ji_dict.values()))

ГЕРБ 0.6428571428571429
БСП за Болгарию 0.11904761904761905
Объединённые патриоты 0.11904761904761905
Движение за права и свободы 0.11904761904761905
Воля 0.0


In [235]:
preference_profiles = [
    ['ГЕРБ', 'БСП за Болгарию', 0.1],
    ['ГЕРБ', 'Объединённые патриоты', 0.2],
    ['ГЕРБ', 'Движение за права и свободы', 0.3],
    ['ГЕРБ ', 'Воля ', 0.4],
    ['БСП за Болгарию', 'Объединённые патриоты', 0.15],
    ['БСП за Болгарию', 'Движение за права и свободы', 0.25],
    ['БСП за Болгарию', 'Воля', 0.35],
    ['Объединённые патриоты', 'Движение за права и свободы', 0.32],
    ['Объединённые патриоты', 'Воля ', 0.58],
    ['Движение за права и свободы', 'Воля', 0.9]
]

In [239]:
winning_coalitions[5]

['ГЕРБ', 'БСП за Болгарию', 'Воля']

In [247]:
[combination for combination in combinations(winning_coalitions[5], 2)]

[('ГЕРБ', 'БСП за Болгарию'), ('ГЕРБ', 'Воля'), ('БСП за Болгарию', 'Воля')]

In [274]:
def min_pref(coalition):
    # print(coalition)
    pairs = [combination for combination in combinations(coalition, 2)]
    prefs = []
    for pair in pairs:
        # print(pair)
        for profile in preference_profiles:
            # print(profile)
            if pair[0] and pair[1] in profile:
                # print(profile[2])
                prefs.append(profile[2])
    return min(prefs)

In [275]:
print(min_pref(winning_coalitions[13]))

0.15


In [281]:
indices = []

for party in list(election_dict.keys()):
    xi_sum = 0
    for coalition in winning_coalitions:
        if party in coalition and is_pivotal(party, coalition):
            xi_sum += min_pref(coalition)
    indices.append([party, xi_sum])
    
print([[index[0], index[1]/sum([index[1] for index in indices])] for index in indices])

[['ГЕРБ', 0.47222222222222215], ['БСП за Болгарию', 0.13888888888888887], ['Объединённые патриоты', 0.16666666666666663], ['Движение за права и свободы', 0.2222222222222222], ['Воля', 0.0]]


In [279]:
indices

[['ГЕРБ', 1.7],
 ['БСП за Болгарию', 0.5],
 ['Объединённые патриоты', 0.6],
 ['Движение за права и свободы', 0.8],
 ['Воля', 0]]

In [287]:
aleskerov = {
    'ГЕРБ': 0.47222222222222215,
    'БСП за Болгарию': 0.13888888888888887,
    'Объединённые патриоты': 0.16666666666666663,
    'Движение за права и свободы': 0.16666666666666663,
    'Воля': 0
}

In [288]:
banzhaf = {
    'ГЕРБ': 0.5,
    'БСП за Болгарию': 0.16666666,
    'Объединённые патриоты': 0.16666666,
    'Движение за права и свободы': 0.16666666,
    'Воля': 0
}

In [291]:
for party in list(election_dict.keys()):
    print(party, (aleskerov[party]/banzhaf[party])*100)

ГЕРБ 94.44444444444443
БСП за Болгарию 83.3333366666668
Объединённые патриоты 100.00000400000013
Движение за права и свободы 100.00000400000013


ZeroDivisionError: division by zero

In [292]:
def digen_packel(party, winning_coalitions):
    party_dp = 0
    for coalition in winning_coalitions:
        if party in coalition and is_pivotal(party, coalition) and is_min_winning(coalition):
            party_dp += 1/sum([is_pivotal(party, coalition) for party in coalition])
    return(party, party_dp)

In [293]:
dp_dict = {digen_packel(party, winning_coalitions)[0]: digen_packel(party, winning_coalitions)[1] for party in list(election_dict.keys())}

In [294]:
dp_dict

{'ГЕРБ': 1.5,
 'БСП за Болгарию': 0.8333333333333333,
 'Объединённые патриоты': 0.8333333333333333,
 'Движение за права и свободы': 0.8333333333333333,
 'Воля': 0}

In [295]:
for party in list(dp_dict.keys()):
    print(party, dp_dict[party]/sum(dp_dict.values()))

ГЕРБ 0.3750000000000001
БСП за Болгарию 0.20833333333333337
Объединённые патриоты 0.20833333333333337
Движение за права и свободы 0.20833333333333337
Воля 0.0
